<a href="https://colab.research.google.com/github/mdfahad999/Experiments/blob/main/Research_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain  openai  tiktoken  duckduckgo-search langsmith requests beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
import requests
from bs4 import BeautifulSoup
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.utilities import DuckDuckGoSearchAPIWrapper
import json

openai_key=''

RESULTS_PER_QUESTION = 3
ddg_search = DuckDuckGoSearchAPIWrapper()


def web_search(query: str, num_results: int = RESULTS_PER_QUESTION):
    results = ddg_search.results(query, num_results)
    return [r["link"] for r in results]


# SUMMARY_TEMPLATE = """Summarize the following question based on the given content:
# Question :{question}

# Content:{content}"""  # noqa: E501


# SUMMARY_PROMPT = ChatPromptTemplate.from_template(SUMMARY_TEMPLATE)

SUMMARY_TEMPLATE = """{text}
-----------
Using the above text, answer in short the following question:
> {question}
-----------
if the question cannot be answered using the text, imply summarize the text. Include all factual information, numbers, stats etc if available."""  # noqa: E501
SUMMARY_PROMPT = ChatPromptTemplate.from_template(SUMMARY_TEMPLATE)


def scrape_text(url: str):
    print(f'URL entered:{url}')
    # Send a GET request to the webpage
    try:
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the content of the request with BeautifulSoup
            soup = BeautifulSoup(response.text, "html.parser")

            # Extract all text from the webpage
            page_text = soup.get_text(separator=" ", strip=True)

            # Print the extracted text

            return page_text
        else:
            return f"Failed to retrieve the webpage: Status code {response.status_code}"
    except Exception as e:
        print(e)
        return f"Failed to retrieve the webpage: {e}"


url = "https://blog.langchain.dev/announcing-langsmith/"

#url='https://www.exlservice.com/insights/article/risk-convergence-how-insurers-can-seize-initiative-privacy-cyber-risk-and'


page_content =scrape_text(url)[:10000]

print(page_content[:])

chain = SUMMARY_PROMPT | ChatOpenAI(model="gpt-3.5-turbo",api_key=openai_key) | StrOutputParser()

#question='how insurers can seize initiative privacy cyber-risk'

question='what is the langsmith all about'


scrape_and_summarize_chain = RunnablePassthrough.assign(
    summary = RunnablePassthrough.assign(
    text=lambda x: scrape_text(x["url"])[:10000]
) | SUMMARY_PROMPT | ChatOpenAI(model="gpt-3.5-turbo",api_key=openai_key) | StrOutputParser()
) | (lambda x: f"\n\n SUMMARY: {x['summary']}\n\n REFERENCE URL: {x['url']}")



web_search_chain = RunnablePassthrough.assign(
    urls = lambda x: web_search(x["question"])
) | (lambda x: [{"question": x["question"], "url": u} for u in x["urls"]]) | scrape_and_summarize_chain.map()


##Adding prompt for generating 3 question about the original query and then curreating all in the information in one report in markdown format


SEARCH_PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            "Write 3 google search queries to search online that form an "
            "objective opinion from the following: {question}\n"
            "You must respond with a list of strings in the following format: "
            '["query 1", "query 2", "query 3"].',
        ),
    ]
)

search_question_chain = SEARCH_PROMPT | ChatOpenAI(model="gpt-3.5-turbo",api_key=openai_key, temperature=0) | StrOutputParser() | json.loads

full_research_chain = search_question_chain | (lambda x: [{"question": q} for q in x]) | web_search_chain.map()

WRITER_SYSTEM_PROMPT = "You are an AI critical thinker research assistant. Your sole purpose is to write well written, critically acclaimed, objective and structured reports on given text."  # noqa: E501


# Report prompts from https://github.com/assafelovic/gpt-researcher/blob/master/gpt_researcher/master/prompts.py
RESEARCH_REPORT_TEMPLATE = """Information:
--------
{research_summary}
--------
Using the above information, answer the following question or topic: "{question}" in a detailed report -- \
The report should focus on the answer to the question, should be well structured, informative, \
in depth, with facts and numbers if available and a minimum of 1,200 words.
You should strive to write the report as long as you can using all relevant and necessary information provided.
You must write the report with markdown syntax.
You MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions.
Write all used source urls at the end of the report, and make sure to not add duplicated sources, but only one reference for each.
You must write the report in apa format.
Please do your best, this is very important to my career."""  # noqa: E501

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", WRITER_SYSTEM_PROMPT),
        ("user", RESEARCH_REPORT_TEMPLATE),
    ]
)

def collapse_list_of_lists(list_of_lists):
    content = []
    for l in list_of_lists:
        content.append("\n\n".join(l))
    return "\n\n".join(content)

chain = RunnablePassthrough.assign(
    research_summary= full_research_chain | collapse_list_of_lists
) | prompt | ChatOpenAI(model="gpt-3.5-turbo",api_key=openai_key) | StrOutputParser()



# result=chain.invoke(
#     {'question':question,
#      }
#     )

# result=web_search_chain.invoke(
#     {'question':question,
#      }
#     )


URL entered:https://blog.langchain.dev/announcing-langsmith/
Announcing LangSmith, a unified platform for debugging, testing, evaluating, and monitoring your LLM applications Skip to content LangChain Blog Home By LangChain Release Notes GitHub Docs Case Studies Sign in Subscribe Announcing LangSmith, a unified platform for debugging, testing, evaluating, and monitoring your LLM applications 8 min read Jul 18, 2023 LangChain exists to make it as easy as possible to develop LLM-powered applications. We started with an open-source Python package when the main blocker for building LLM-powered applications was getting a simple prototype working. We remember seeing Nat Friedman tweet in late 2022 that there was “ not enough tinkering happening .” The LangChain open-source packages are aimed at addressing this and we see lots of tinkering happening now ( Nat agrees )–people are building everything from chatbots over internal company documents to an AI dungeon master for a Dungeons and Dragon

In [ ]:
print(result )

["\n\n SUMMARY: LangSmith is a new product by the creators of LangChain, aimed at helping developers get their large language models (LLM’s) into production in a reliable and maintainable way. It focuses on solving production challenges around reliability, debugging, testing, evaluating, monitoring, and usage metrics. The goal is to provide a simple and intuitive UI to reduce the barrier to entry for developers without a software background. LangSmith aims to be differentiated by offering extensibility and integrations with other tools, while also connecting with as many tools as possible. Competitors in this space include Vercel, AI SDK providers, and Embeddings providers. LangSmith's long-term success will depend on its ability to continue expanding in scope and remaining competitive with multiple providers and other tooling ecosystems.\n\n REFERENCE URL: https://logankilpatrick.medium.com/what-is-langsmith-and-why-should-i-care-as-a-developer-e5921deb54b5", '\n\n SUMMARY: LangSmith 

In [ ]:
import json

# Given string representation of a list
string_representation = '["Langchain vs Langsmith differences"]'

# Convert the string to a Python list
python_list = json.loads(string_representation.replace("'", '"'))

# Print the Python list
print("Python List:", python_list)

# Convert the Python list to JSON
json_representation = json.dumps(python_list)

# Print the JSON representation
print("JSON Representation:", json_representation)

Python List: ['Langchain vs Langsmith differences']
JSON Representation: ["Langchain vs Langsmith differences"]


In [ ]:
#!pip install gradio

In [ ]:
import gradio as gr

def Research_assistant(question):


    scrape_and_summarize_chain = RunnablePassthrough.assign(
        summary = RunnablePassthrough.assign(
        text=lambda x: scrape_text(x["url"])[:10000]
    ) | SUMMARY_PROMPT | ChatOpenAI(model="gpt-3.5-turbo",api_key=openai_key) | StrOutputParser()
    ) | (lambda x: f"\n\n SUMMARY: {x['summary']}\n\n REFERENCE URL: {x['url']}")



    web_search_chain = RunnablePassthrough.assign(
        urls = lambda x: web_search(x["question"])
    ) | (lambda x: [{"question": x["question"], "url": u} for u in x["urls"]]) | scrape_and_summarize_chain.map()



    # result=web_search_chain.invoke(
    #     {'question':question,
    #     }
    #     )
    #print('Type of the result', type(result))

    ##Adding prompt for generating 3 question about the original query and then curreating all in the information in one report in markdown format

    SEARCH_PROMPT = ChatPromptTemplate.from_messages(
        [
            (
                "user",
                "Write 3 google search queries to search online that form an "
                "objective opinion from the following: {question}\n"
                "You must strictly  respond with a list and convert to strings in the following format: "
                """```["query 1", "query 2", "query 3"]
                    ```
                    Everything between the ``` must be valid array.""",
            ),
        ]
    )
    def make_questions(questions):


        print(f'Questions:: {questions}')
        # result_list = []
        # result_list.extend(questions)
        data=json.loads(str(questions))
        print('Done loading')
        return data


    search_question_chain = SEARCH_PROMPT |ChatOpenAI(model="gpt-3.5-turbo",api_key=openai_key) | StrOutputParser() | make_questions

    # def make_questions(questions):
    #     print(f'Questions::'  {questions})
    #     new=[]
    #     for q in questions:
    #         new.append({"question":q})
    #     return new

    full_research_chain = search_question_chain | (lambda x: [{"question": q} for q in x]) | web_search_chain.map()
    #full_research_chain = search_question_chain | make_questions | web_search_chain.map()

    WRITER_SYSTEM_PROMPT = "You are an AI critical thinker research assistant. Your sole purpose is to write well written, critically acclaimed, objective and structured reports on given text."  # noqa: E501


    # Report prompts from https://github.com/assafelovic/gpt-researcher/blob/master/gpt_researcher/master/prompts.py
    RESEARCH_REPORT_TEMPLATE = """Information:
    --------
    {research_summary}
    --------
    Using the above information, answer the following question or topic: "{question}" in a detailed report -- \
    The report should focus on the answer to the question, should be well structured, informative, \
    in depth, with facts and numbers if available and a minimum of 1,200 words.
    You should strive to write the report as long as you can using all relevant and necessary information provided.
    You must write the report with markdown syntax.
    You MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions.
    Write all used source urls at the end of the report, and make sure to not add duplicated sources, but only one reference for each.
    You must write the report in apa format.
    Please do your best, this is very important to my career."""  # noqa: E501

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", WRITER_SYSTEM_PROMPT),
            ("user", RESEARCH_REPORT_TEMPLATE),
        ]
    )

    def collapse_list_of_lists(list_of_lists):
        content = []
        for l in list_of_lists:
            content.append("\n\n".join(l))
        return "\n\n".join(content)

    chain = RunnablePassthrough.assign(
        research_summary= full_research_chain | collapse_list_of_lists
    ) | prompt | ChatOpenAI(model="gpt-3.5-turbo",api_key=openai_key) | StrOutputParser()
    print('RESEARCH_REPORT_TEMPLATE completed')


    result=chain.invoke(
        {'question':question,
        }
        )
    print('Type of the result', type(result))

    if isinstance(result, list):

        return ''.join(result)
    else:

        return result

demo = gr.Interface(
    fn=Research_assistant,
    inputs=gr.Textbox(lines=2, placeholder="Enter your text here bro!!!"),
    outputs="text",
)
demo.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8fe4333691ea202534.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


RESEARCH_REPORT_TEMPLATE completed
Questions:: ["Pros and cons of Meta Glasses", "Advantages and disadvantages of Meta Glasses", "Meta Glasses: benefits and drawbacks"]
Done loading
URL entered:https://www.wired.com/review/review-ray-ban-meta-smart-glasses/
URL entered:https://www.wired.com/review/review-ray-ban-meta-smart-glasses/
URL entered:https://www.tomsguide.com/reviews/ray-ban-meta-smart-glasses
URL entered:https://www.howtogeek.com/ray-ban-meta-smart-glasses-review/URL entered:https://www.wired.com/review/review-ray-ban-meta-smart-glasses/

URL entered:https://www.tomsguide.com/reviews/ray-ban-meta-smart-glasses
URL entered:https://www.tomsguide.com/reviews/ray-ban-meta-smart-glasses
URL entered:https://www.theverge.com/23922425/ray-ban-meta-smart-glasses-review
URL entered:https://www.theverge.com/23922425/ray-ban-meta-smart-glasses-review
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Type of the result <class 'str'>
RESEARCH_REP